In [15]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
torch.manual_seed(17)
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [16]:
# Download dataset and prepare dataloaders
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
DEVICE = 'cpu'

train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

full_train_dataset = datasets.CIFAR10(root='data', train=True, transform=train_transforms, download=True)
train_dataset, validation_dataset = torch.utils.data.random_split(full_train_dataset, [40000, 10000])

test_dataset = datasets.CIFAR10(root='data', train=False, transform=test_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, num_workers=2, shuffle=False)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified


In [ ]:
# Train model
def train(model, weight_decay=0):
    # Define loss function
    criterion = nn.CrossEntropyLoss()
    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), weight_decay=weight_decay)

    # Store losses to plot after training finishes
    train_losses = []
    validation_losses = []

    for epoch in range(1, NUM_EPOCHS+1):
        # Track training + validation loss
        train_loss = 0.0
        validation_loss = 0.0
        
        # Train the model
        model.train()
        for data, target in train_loader:
            # Clear gradients
            optimizer.zero_grad()
            # Forward pass - compute predictions by passing input through model
            output = model(data)
            # Calculate loss
            loss = criterion(output, target)
            # Backpropogation: compute gradient of loss w/ respect to model parameters
            loss.backward()
            # Backpropogation: Update parameters using loss gradient
            optimizer.step()
            # Update train loss
            train_loss += loss.item()*data.size(0)
            
        # Check accuracy on validation set to make sure we don't overfit
        model.eval()
        for data, target in validation_loader:
            # Forward pass - compute predictions by passing input through model
            output = model(data)
            # Calculate loss
            loss = criterion(output, target)
            # Update validation loss
            validation_loss += loss.item()*data.size(0)
        
        # Calculate average train and validation losses
        train_loss = train_loss/len(train_loader.dataset)
        validation_loss = validation_loss/len(validation_loader.dataset)
        train_losses.append(train_loss)
        validation_losses.append(validation_loss)
            
        # Display training and validation loss and accuracy every epoch 
        train_accuracy = get_accuracy(model, train_loader, DEVICE)
        validation_accuracy = get_accuracy(model, validation_loader, DEVICE)
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
            epoch, train_loss, validation_loss, train_accuracy, validation_accuracy))
    return train_losses, validation_losses